# What this notebook does

### Title: LLM-scCurator Colab Quickstart (Python)

Scanpy demo (public PBMC3k): run clustering in Python, then curate markers and optionally annotate clusters with an LLM.

Core requirement:
- LLM-scCurator expects log1p-normalized expression in `adata.X`.
  Keep raw counts in `adata.layers["counts"]`.


## Install Python packages + LLM-scCurator

In [ ]:
%pip install -q -U google-generativeai==0.8.5 scanpy==1.11.5 pandas==2.3.3 scipy==1.16.3
%pip -q install llm-sc-curator

## Imports + reproducibility

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import random, os

import google.generativeai as genai
import llm_sc_curator
from llm_sc_curator import LLMscCurator
from llm_sc_curator.masking import FeatureDistiller
from llm_sc_curator.backends import OpenAIBackend

import warnings
warnings.filterwarnings("ignore")

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"🔒 Random seed set to {RANDOM_SEED} for reproducibility.")

## Data input: Scanpy demo PBMC3k

In [ ]:
# Load demo data
adata = sc.datasets.pbmc3k()

# Minimal QC
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata

In [ ]:
# Save raw counts, then normalize/log1p into .X (required by LLM-scCurator)
adata.layers["counts"] = adata.X.copy()

# Normalize + log1p (standard)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# HVG selection
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    flavor="seurat",
    subset=False,
)
sc.pl.highly_variable_genes(adata)

In [ ]:
# PCA/Neighbors/UMAP/Leiden using HVGs
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)
sc.pl.umap(adata, color=["leiden"])


##  Run LLM-scCurator, noise masking and Gini-informed distillation

In [ ]:
ACCURACY_MODE = False  # set False for cheaper/faster

MODEL_NAME = "gpt-5.2" if ACCURACY_MODE else "gpt-4o"
print("Using model:", MODEL_NAME)

In [ ]:
# Recommended: store your key in Colab Secrets as "GEMINI_API_KEY"
from google.colab import userdata
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY") or "YOUR_KEY_HERE"

In [ ]:
# Curator Initialize
from llm_sc_curator.backends import OpenAIBackend

backend = OpenAIBackend(api_key=OPENAI_API_KEY, model_name=MODEL_NAME)
curator = LLMscCurator(backend=backend)
curator.set_global_context(adata)

In [ ]:
# Choose a cluster (edit if you want a specific one)
cluster_name = sorted(adata.obs["leiden"].astype(str).unique())[0]
print(f"cluster_name = {cluster_name}")

In [ ]:
# --- Standard DE: Target vs Rest ---
N_GENES = 50
TMP = "__tmp_binary__"

# --- Standard: DE (Target vs Rest) ---
adata.obs[TMP] = "Rest"
adata.obs.loc[adata.obs["leiden"].astype(str) == str(cluster_name), TMP] = "Target"

sc.tl.rank_genes_groups(
    adata,
    groupby=TMP,
    groups=["Target"],
    reference="Rest",
    method="wilcoxon",
    use_raw=False,
)

df_std = sc.get.rank_genes_groups_df(adata, group="Target")
genes_std = df_std["names"].head(N_GENES).tolist()

adata.obs.drop(columns=[TMP], inplace=True, errors="ignore")

# --- Curated: LLM-scCurator ---
genes_cur = curator.curate_features(
    adata,
    group_col="leiden",
    target_group=str(cluster_name),
    n_top=N_GENES,
    use_statistics=True,
)

print("genes_std len =", len(genes_std))
print("genes_cur len =", len(genes_cur))


In [ ]:
# --- Readable output: lists + rescued/dropped + table ---
def _fmt_list(title, genes, n=10, bullet="•"):
    genes = [str(g) for g in (genes or [])]
    head = genes[:n]
    body = "\n".join([f"{bullet} {g}" for g in head]) if head else f"{bullet} (empty)"
    return f"{title} (top {len(genes)}, showing {min(n, len(genes))})\n{body}"

std = [str(g) for g in genes_std]
cur = [str(g) for g in genes_cur]

std_set = set(std)
cur_set = set(cur)

rescued = [g for g in cur if g not in std_set]
dropped = [g for g in std if g not in cur_set]
overlap = [g for g in cur if g in std_set]

print(f"\nGene list summary: std={len(std)} | curated={len(cur)} | shared={len(overlap)} | rescued={len(rescued)} | dropped={len(dropped)}\n")

print("🟦 " + _fmt_list("Standard genes", std, n=10))
print()
print("🟥 " + _fmt_list("Curated genes", cur, n=10))
print()
print("➕ " + _fmt_list("Rescued (curated-only)", rescued, n=10))
print()
print("➖ " + _fmt_list("Dropped (std-only)", dropped, n=10))

TOP = 15
def _pad(xs, n=TOP):
    xs = [str(x) for x in (xs or [])][:n]
    return xs + [""] * (n - len(xs))

df_view = pd.DataFrame({
    "🏆 rank ": list(range(1, TOP + 1)),
    "🟦 standard ": _pad(genes_std, TOP),
    "🟥 curated ":  _pad(genes_cur, TOP),
})

df_view.insert(0, "cluster", str(cluster_name))
df_view

## (Optinal) Run LLM-scCurator hierarchical discovery (writes major_type / fine_type)

In [ ]:
# ============================================================
# Optional: automatic hierarchical annotation (coarse-to-fine)
# ============================================================

# This will add columns like adata.obs["major_type"] and adata.obs["fine_type"].
%%time

# Curator Initialize
curator = LLMscCurator(backend=backend)

N_GENES = 50

import time
t0 = time.time()
adata_llm = curator.run_hierarchical_discovery(
    adata,
    n_top=N_GENES,
    batch_key=None, # set this if you want batch-aware behavior
)
t1 = time.time()

print(f"run_hierarchical_discovery() total: {t1 - t0:.1f} sec")
display(adata_llm.obs[["major_type","fine_type"]].head())


In [ ]:
# Plot only if they exist
cols_to_plot = [c for c in ["major_type", "fine_type"] if c in adata.obs.columns]
if cols_to_plot and "X_umap" in adata.obsm:
    sc.pl.umap(adata, color=cols_to_plot, ncols=1)

## Quick marker dotplots for sanity-checking labels

In [ ]:
sc.set_figure_params(dpi=140, fontsize=12)

# ---------- marker panels (canonical) ----------
MARKERS_MAJOR = {
    "T-lineage":       ["CD3D", "CD3E", "LTB", "IL7R", "CD8A","TRAC"],
    "NK/Cytotoxic":    ["NKG7", "GNLY", "PRF1", "GZMB", "FCER1G"],
    "B cell":          ["MS4A1", "CD79A", "CD74", "HLA-DRA", "CD37"],
    "Monocyte":        ["LYZ", "LST1", "S100A8", "S100A9", "FCGR3A", "MS4A7"],
    "Megakaryocyte":   ["PPBP", "PF4", "ITGA2B", "GP9", "SDPR"],
}

# ---------- Utility: drop genes not present + dedupe across groups ----------
def filter_and_dedupe_markers(marker_dict, var_names):
    vset = set(map(str, var_names))
    used = set()
    out = {}
    for group, genes in marker_dict.items():
        cleaned = []
        for g in genes:
            g = str(g)
            if g not in vset:
                continue
            if g in used:
                continue
            cleaned.append(g)
            used.add(g)
        if cleaned:
            out[group] = cleaned
    return out

# Major type dotplot
markers = filter_and_dedupe_markers(MARKERS_MAJOR, adata.var_names)
sc.pl.dotplot(
    adata,
    markers,
    groupby="major_type",
    standard_scale="var",
    use_raw=False,
    swap_axes=True,
    dot_min=0.0,
    dot_max=0.8,
    show=True,
)





In [ ]:
sc.pl.umap(adata_llm,
           color=["major_type",
                  "fine_type",
                  MARKERS_MAJOR["T-lineage"][0],
                  MARKERS_MAJOR["NK/Cytotoxic"][0],
                  MARKERS_MAJOR["B cell"][0],
                  MARKERS_MAJOR["Monocyte"][0],
                  ]+
                  MARKERS_MAJOR["Megakaryocyte"][:2],

           ncols=2,
           use_raw=False
           )


In [ ]:
MARKERS_FINE = {
    "T-lineage":         ["TRAC", "CD3D", "CD3E", "CD3G", "LTB", "IL7R"],
    "CD8 / activation":  ["CD8A", "CD8B", "CISH", "CCR7", "MAL"],
    "Cytotoxic":         ["NKG7", "GNLY", "PRF1", "GZMB", "CCL5"],
    "Proliferation":     ["MKI67", "TOP2A", "STMN1", "HMGB2", "TYMS"],
    "Classical mono":    ["S100A8", "S100A9", "LYZ", "LST1", "CTSS"],
    "FCGR3A+ mono":      ["FCGR3A", "MS4A7", "IFITM3", "LGALS3BP"],
    "HLA-II / APC":      ["HLA-DRA", "HLA-DRB1", "HLA-DPA1", "HLA-DPB1", "CD74"],
    "B cell":            ["MS4A1", "CD79A", "CD74", "CD37"],
    "Dendritic (FCER1A+)": ["FCER1A", "CLEC10A"],
    "Platelet/Megakaryocyte": ["PPBP", "PF4", "ITGA2B", "GP9", "TUBB1"],
}



# ---------- Plot settings ----------
markers = filter_and_dedupe_markers(MARKERS_FINE, adata.var_names)
sc.pl.dotplot(
    adata,
    markers,
    groupby="fine_type",
    standard_scale="var",
    use_raw=False,
    swap_axes=True,
    dot_min=0.0,
    dot_max=0.8,
    show=True,
)
